In [7]:
# ============================================================================
# 🚀 UIDAI AADHAAR INTELLIGENCE SYSTEM - COMPLETE ML PIPELINE
# ============================================================================
# Single Google Colab Notebook - All-in-One Solution
# Detects Anomalies, Patterns, Trends & Generates Policy Indexes
# ============================================================================

# STEP 0: INSTALL & IMPORT LIBRARIES
# ============================================================================

print("="*80)
print("📦 INSTALLING REQUIRED LIBRARIES...")
print("="*80)

!pip install pandas numpy scikit-learn scipy matplotlib seaborn openpyxl -q

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import IsolationForest
from scipy import stats
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("✅ All libraries installed and imported successfully!\n")


📦 INSTALLING REQUIRED LIBRARIES...
✅ All libraries installed and imported successfully!



In [8]:

# ============================================================================
# STEP 1: UPLOAD & LOAD DATA
# ============================================================================

print("="*80)
print("📂 STEP 1: DATA LOADING")
print("="*80)

# Load file directly from Colab sidebar (no upload needed!)
# Make sure 'merged_aadhaar_data.csv' is in the Files section on the left sidebar
print("\n📂 Loading file from Colab sidebar...")

# Try to load the file
try:
    df = pd.read_csv('merged_aadhaar_data.csv')
    print("✅ File loaded successfully from sidebar!")
except FileNotFoundError:
    print("❌ File not found in sidebar!")
    print("   Please make sure 'merged_aadhaar_data.csv' is uploaded to the sidebar.")
    print("   (Click the 📁 folder icon on the left, then upload your file)")
    raise

print(f"\n✅ Data loaded successfully!")
print(f"   Total rows: {len(df):,}")
print(f"   Columns: {df.columns.tolist()}")
print(f"\n📊 First 5 rows:")
print(df.head())

📂 STEP 1: DATA LOADING

📂 Loading file from Colab sidebar...
❌ File not found in sidebar!
   Please make sure 'merged_aadhaar_data.csv' is uploaded to the sidebar.
   (Click the 📁 folder icon on the left, then upload your file)


FileNotFoundError: [Errno 2] No such file or directory: 'merged_aadhaar_data.csv'

In [ ]:
# ============================================================================
# STEP 2: DATA PREPARATION
# ============================================================================

print("\n" + "="*80)
print("🔧 STEP 2: DATA PREPARATION")
print("="*80)

# Convert Year_Month to datetime
df['Year_Month'] = pd.to_datetime(df['Year_Month'], format='%Y-%m', errors='coerce')

# Add time features
df['Year'] = df['Year_Month'].dt.year
df['Month'] = df['Year_Month'].dt.month
df['Month_Name'] = df['Year_Month'].dt.strftime('%B')

# Remove missing values
df = df.dropna(subset=['Count', 'State', 'District'])

# Sort by date
df = df.sort_values('Year_Month')

print(f"✅ Data prepared")
print(f"   Date range: {df['Year_Month'].min()} to {df['Year_Month'].max()}")
print(f"   Total records: {len(df):,}")


In [ ]:
# ============================================================================
# STEP 3: BASELINE CALCULATION (WHAT'S NORMAL?)
# ============================================================================

print("\n" + "="*80)
print("📊 STEP 3: CALCULATING BASELINES (6-MONTH ROLLING AVERAGE)")
print("="*80)

baseline_data = []

# For each group (State, District, Metric_Type, Age_Group)
for (state, district, metric_type, age_group), group in df.groupby(['State', 'District', 'Metric_Type', 'Age_Group']):
    group = group.sort_values('Year_Month').copy()

    # Calculate 6-month rolling statistics
    group['Baseline_Mean'] = group['Count'].rolling(window=6, min_periods=3).mean()
    group['Baseline_Std'] = group['Count'].rolling(window=6, min_periods=3).std()

    # Fill NaN with overall statistics
    group['Baseline_Mean'].fillna(group['Count'].mean(), inplace=True)
    group['Baseline_Std'].fillna(group['Count'].std() if group['Count'].std() > 0 else 1, inplace=True)

    # Calculate deviations
    group['Deviation'] = group['Count'] - group['Baseline_Mean']
    group['Deviation_Pct'] = (group['Deviation'] / group['Baseline_Mean']) * 100
    group['Z_Score'] = np.where(
        group['Baseline_Std'] > 0,
        group['Deviation'] / group['Baseline_Std'],
        0
    )

    baseline_data.append(group)

df = pd.concat(baseline_data, ignore_index=True)

print(f"✅ Baselines calculated for all {len(df):,} records")


In [ ]:

# ============================================================================
# STEP 4: ANOMALY DETECTION (ENSEMBLE: Z-SCORE + IQR + ISOLATION FOREST)
# ============================================================================
from sklearn.ensemble import IsolationForest
import pandas as pd
import numpy as np

print("\n" + "="*80)
print("🚨 STEP 4: ANOMALY DETECTION (RECTIFIED ENSEMBLE)")
print("="*80)

# ----------------------------------------------------------------------------
# INITIALIZE COLUMNS
# ----------------------------------------------------------------------------
df['Anomaly_ZScore'] = False
df['Anomaly_IQR'] = False
df['Anomaly_IsoForest'] = False
df['Anomaly_Ensemble'] = False
df['Anomaly_Severity'] = 0.0

# ----------------------------------------------------------------------------
# 1️⃣ Z-SCORE + IQR (GROUP-WISE – FAST & SAFE)
# ----------------------------------------------------------------------------
for (state, district, metric_type, age_group), group in df.groupby(
    ['State', 'District', 'Metric_Type', 'Age_Group']
):
    idx = group.index

    # Z-SCORE (threshold = 2.5)
    df.loc[idx, 'Anomaly_ZScore'] = group['Z_Score'].abs() > 2.5

    # IQR (only if enough data)
    if len(group) >= 4:
        Q1 = group['Count'].quantile(0.25)
        Q3 = group['Count'].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR

        df.loc[idx, 'Anomaly_IQR'] = (
            (group['Count'] < lower) |
            (group['Count'] > upper)
        )

# ----------------------------------------------------------------------------
# 2️⃣ ISOLATION FOREST (METRIC + AGE LEVEL — NOT DISTRICT LEVEL)
# ----------------------------------------------------------------------------
for (metric_type, age_group), group in df.groupby(['Metric_Type', 'Age_Group']):

    # Minimum data requirement
    if len(group) < 50:
        continue

    iso_model = IsolationForest(
        contamination=0.1,
        n_estimators=50,
        max_samples='auto',
        random_state=42,
        n_jobs=-1
    )

    X = group[['Count']].values
    preds = iso_model.fit_predict(X)

    df.loc[group.index, 'Anomaly_IsoForest'] = (preds == -1)

# ----------------------------------------------------------------------------
# 3️⃣ ENSEMBLE VOTING (2 OUT OF 3)
# ----------------------------------------------------------------------------
df['Anomaly_Ensemble'] = (
    df['Anomaly_ZScore'].astype(int) +
    df['Anomaly_IQR'].astype(int) +
    df['Anomaly_IsoForest'].astype(int)
) >= 2

# ----------------------------------------------------------------------------
# 4️⃣ SEVERITY SCORE (0 → 1)
# ----------------------------------------------------------------------------
df['Anomaly_Severity'] = (
    df['Z_Score'].abs() / 5
).clip(0, 1)

# ----------------------------------------------------------------------------
# 5️⃣ SEVERITY CLASSIFICATION
# ----------------------------------------------------------------------------
df['Severity_Level'] = pd.cut(
    df['Anomaly_Severity'],
    bins=[0, 0.4, 0.7, 1.0],
    labels=['MEDIUM', 'HIGH', 'CRITICAL'],
    include_lowest=True
)

# ----------------------------------------------------------------------------
# 6️⃣ SUMMARY
# ----------------------------------------------------------------------------
anomaly_count = int(df['Anomaly_Ensemble'].sum())
total_rows = len(df)

print("✅ Anomaly detection complete")
print(f"   Total anomalies: {anomaly_count:,}")
print(f"   Anomaly rate: {(anomaly_count / total_rows * 100):.2f}%")

print("\n   By Severity:")
severity_counts = (
    df[df['Anomaly_Ensemble']]
    ['Severity_Level']
    .value_counts()
)

for level, count in severity_counts.items():
    print(f"      {level}: {count:,}")


In [ ]:
# ============================================================================
# STEP 5: PATTERN DETECTION (SEASONAL TRENDS)
# ============================================================================

print("\n" + "="*80)
print("🔄 STEP 5: PATTERN DETECTION")
print("="*80)

df['Pattern_Seasonal'] = False
df['Pattern_Type'] = 'NONE'

pattern_count = 0

for (state, district, metric_type, age_group), group in df.groupby(['State', 'District', 'Metric_Type', 'Age_Group']):
    if len(group) < 12:
        continue

    indices = group.index

    # Calculate monthly averages
    monthly_avg = group.groupby('Month')['Count'].mean()
    cv = monthly_avg.std() / monthly_avg.mean() if monthly_avg.mean() > 0 else 0

    # Detect seasonal pattern (CV > 20%)
    has_pattern = cv > 0.2

    if has_pattern:
        df.loc[indices, 'Pattern_Seasonal'] = True
        df.loc[indices, 'Pattern_Type'] = 'SEASONAL_SPIKE'
        pattern_count += len(indices)

print(f"✅ Pattern detection complete")
print(f"   Seasonal patterns detected: {pattern_count:,} records")



In [ ]:
# ============================================================================
# STEP 6: TREND DETECTION (LINEAR REGRESSION)
# ============================================================================

print("\n" + "="*80)
print("📈 STEP 6: TREND DETECTION")
print("="*80)

df['Trend_Direction'] = 'STABLE'
df['Trend_Slope'] = 0.0
df['Trend_RSquared'] = 0.0

trend_count = 0

for (state, district, metric_type, age_group), group in df.groupby(['State', 'District', 'Metric_Type', 'Age_Group']):
    if len(group) < 6:
        continue

    indices = group.index

    # Linear regression
    X = np.arange(len(group))
    y = group['Count'].values

    slope, intercept, r_value, p_value, std_err = stats.linregress(X, y)
    r_squared = r_value ** 2

    # Significant trend: R² > 0.5 and p < 0.05
    if r_squared > 0.5 and p_value < 0.05:
        if slope > 0:
            direction = 'INCREASING'
        else:
            direction = 'DECREASING'

        df.loc[indices, 'Trend_Direction'] = direction
        df.loc[indices, 'Trend_Slope'] = slope
        df.loc[indices, 'Trend_RSquared'] = r_squared
        trend_count += len(indices)

print(f"✅ Trend detection complete")
print(f"   Trends detected: {trend_count:,} records")



In [ ]:
# ============================================================================
# STEP 7: POLICY INDEXES (DPI, OSI, UAG, COMPOSITE RISK)
# ============================================================================

print("\n" + "="*80)
print("📊 STEP 7: CALCULATING POLICY INDEXES")
print("="*80)

# DPI (Demand Pressure Index) - 0 to 10 scale
df['DPI'] = (abs(df['Deviation_Pct']) / 10).clip(0, 10)

# OSI (Operational Stress Index) - based on volatility
df['OSI'] = ((df['Baseline_Std'] / df['Baseline_Mean']) * 10).clip(0, 10)
df['OSI'].replace([np.inf, -np.inf], 0, inplace=True)
df['OSI'].fillna(0, inplace=True)

# UAG (Update Accessibility Gap) - for update metrics only
df['UAG'] = 0.0
update_mask = df['Metric_Type'].isin(['BIOMETRIC_UPDATE', 'DEMOGRAPHIC_UPDATE'])
df.loc[update_mask & (df['Baseline_Mean'] > 0), 'UAG'] = (
    ((df.loc[update_mask & (df['Baseline_Mean'] > 0), 'Baseline_Mean'] -
      df.loc[update_mask & (df['Baseline_Mean'] > 0), 'Count']) /
     df.loc[update_mask & (df['Baseline_Mean'] > 0), 'Baseline_Mean']) * 10
).clip(0, 10)

# Composite Risk Score (weighted average)
df['Composite_Risk'] = (0.4 * df['DPI'] + 0.3 * df['OSI'] + 0.3 * df['UAG'])

# Priority classification
df['Priority'] = pd.cut(
    df['Composite_Risk'],
    bins=[0, 3, 5, 7, 10],
    labels=['LOW', 'MEDIUM', 'HIGH', 'CRITICAL']
)

print(f"✅ Policy indexes calculated")



In [ ]:
# ============================================================================
# STEP 8: SOLUTION FRAMEWORK MAPPING
# ============================================================================

print("\n" + "="*80)
print("💡 STEP 8: SOLUTION FRAMEWORK MAPPING")
print("="*80)

df['Solution_Framework'] = 'MONITOR_ONLY'
df['Recommended_Actions'] = 'Continue monitoring'

# Map frameworks based on indexes
for idx, row in df.iterrows():
    frameworks = []

    if row['DPI'] > 6:
        frameworks.append('CAPACITY_AUGMENTATION')
    if row['OSI'] > 6:
        frameworks.append('OPERATIONAL_STABILIZATION')
    if row['UAG'] > 6:
        frameworks.append('INCLUSION_OUTREACH')
    if row['Anomaly_Ensemble']:
        frameworks.append('INVESTIGATION')

    if frameworks:
        df.at[idx, 'Solution_Framework'] = ', '.join(frameworks)

print(f"✅ Solution frameworks mapped")



In [ ]:
# ============================================================================
# STEP 9: KEY INSIGHTS & FINDINGS
# ============================================================================
print("\n" + "="*80)
print("💎 STEP 9: EXTRACTING KEY INSIGHTS")
print("="*80)

# ----------------------------------------------------------------------------
# 1️⃣ TOP 10 MOST SHOCKING ANOMALIES
# ----------------------------------------------------------------------------

# Ensure Year_Month is datetime (defensive)
df['Year_Month'] = pd.to_datetime(df['Year_Month'], errors='coerce')

top_anomalies = (
    df[df['Anomaly_Ensemble']]
    .nlargest(10, 'Z_Score')
)

print("\n🔥 TOP 10 MOST SHOCKING ANOMALIES:\n")

if top_anomalies.empty:
    print("   No anomalies detected.")
else:
    for i, (idx, row) in enumerate(top_anomalies.iterrows(), 1):

        direction = "SPIKE ↗️" if row['Deviation_Pct'] > 0 else "DROP ↘️"

        # SAFE DATE FORMATTING
        date_str = (
            row['Year_Month'].strftime('%B %Y')
            if pd.notna(row['Year_Month'])
            else "Unknown Date"
        )

        print(f"{i}. [{row['Severity_Level']}] {row['State']} - {row['District']}")
        print(f"   Date: {date_str}")
        print(f"   Type: {row['Metric_Type']} ({row['Age_Group']})")
        print(f"   {direction} {abs(row['Deviation_Pct']):.1f}% (Z={row['Z_Score']:.2f})")
        print()

# ----------------------------------------------------------------------------
# 2️⃣ GEOGRAPHIC DISTRIBUTION OF ANOMALIES
# ----------------------------------------------------------------------------

print("\n📍 TOP 10 STATES WITH MOST ANOMALIES:\n")

state_anomalies = (
    df[df['Anomaly_Ensemble']]
    .groupby('State')
    .size()
    .sort_values(ascending=False)
    .head(10)
)

if state_anomalies.empty:
    print("   No state-level anomalies found.")
else:
    for state, count in state_anomalies.items():
        print(f"   {state}: {count:,}")

# ----------------------------------------------------------------------------
# 3️⃣ TEMPORAL PATTERNS (MONTH-WISE)
# ----------------------------------------------------------------------------

print("\n📅 ANOMALIES BY MONTH:\n")

month_order = [
    'January', 'February', 'March', 'April', 'May', 'June',
    'July', 'August', 'September', 'October', 'November', 'December'
]

monthly_anomalies = (
    df[df['Anomaly_Ensemble']]
    .groupby('Month_Name')
    .size()
    .reindex(month_order, fill_value=0)
)

for month, count in monthly_anomalies.items():
    bar = "█" * max(1, int(count / 100)) if count > 0 else ""
    print(f"   {month:12s}: {count:5,} {bar}")

# ----------------------------------------------------------------------------
# 4️⃣ TOP RISK DISTRICTS (COMPOSITE RISK)
# ----------------------------------------------------------------------------

print("\n🚨 TOP 10 HIGHEST RISK DISTRICTS:\n")

risk_columns = ['Composite_Risk', 'DPI', 'OSI', 'UAG']

# Defensive check
missing_cols = [c for c in risk_columns if c not in df.columns]
if missing_cols:
    print(f"   ⚠️ Missing risk columns: {missing_cols}")
else:
    top_risk = (
        df.groupby(['State', 'District'])
        .agg({
            'Composite_Risk': 'mean',
            'DPI': 'mean',
            'OSI': 'mean',
            'UAG': 'mean'
        })
        .round(2)
        .nlargest(10, 'Composite_Risk')
    )

    if top_risk.empty:
        print("   No high-risk districts found.")
    else:
        for (state, district), row in top_risk.iterrows():
            print(f"   {district}, {state}")
            print(
                f"      Risk: {row['Composite_Risk']}/10 | "
                f"DPI: {row['DPI']} | "
                f"OSI: {row['OSI']} | "
                f"UAG: {row['UAG']}"
            )


In [ ]:
# ============================================================================
# STEP 10: VISUALIZATIONS
# ============================================================================

# ============================================================================
# STEP 10: VISUALIZATIONS (RECTIFIED & SAFE)
# ============================================================================

print("\n" + "="*80)
print("📊 STEP 10: CREATING VISUALIZATIONS")
print("="*80)

# ----------------------------------------------------------------------------
# Defensive imports & settings
# ----------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# ----------------------------------------------------------------------------
# Ensure required columns exist
# ----------------------------------------------------------------------------
df['Year_Month'] = pd.to_datetime(df['Year_Month'], errors='coerce')

if 'Month' not in df.columns:
    df['Month'] = df['Year_Month'].dt.month

# ----------------------------------------------------------------------------
# 1️⃣ ANOMALY HEATMAP (STATE × MONTH)
# ----------------------------------------------------------------------------
print("\n📈 Creating Anomaly Heatmap...")

anomaly_df = df[df['Anomaly_Ensemble']].copy()

if anomaly_df.empty:
    print("⚠️ No anomalies detected — skipping heatmap.")
else:
    anomaly_pivot = anomaly_df.pivot_table(
        index='State',
        columns='Month',
        values='Anomaly_Ensemble',
        aggfunc='sum',
        fill_value=0
    )

    if anomaly_pivot.empty:
        print("⚠️ Heatmap data empty after pivot — skipping.")
    else:
        fig, ax = plt.subplots(figsize=(14, 8))
        sns.heatmap(
            anomaly_pivot,
            annot=True,
            fmt='g',
            cmap='YlOrRd',
            ax=ax,
            cbar_kws={'label': 'Anomaly Count'}
        )

        ax.set_title('Anomaly Distribution: States × Months', fontsize=16, fontweight='bold')
        ax.set_xlabel('Month', fontsize=12)
        ax.set_ylabel('State', fontsize=12)

        plt.tight_layout()
        plt.savefig('anomaly_heatmap.png', dpi=300, bbox_inches='tight')
        plt.show()

# ----------------------------------------------------------------------------
# 2️⃣ RISK DISTRIBUTION DASHBOARD
# ----------------------------------------------------------------------------
print("\n📈 Creating Risk Distribution Charts...")

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Priority distribution
if 'Priority' in df.columns:
    df['Priority'].value_counts().plot(kind='bar', ax=axes[0, 0], color='coral')
    axes[0, 0].set_title('Cases by Priority Level', fontweight='bold')
    axes[0, 0].set_xlabel('Priority')
    axes[0, 0].set_ylabel('Count')
else:
    axes[0, 0].text(0.5, 0.5, 'Priority data missing', ha='center', va='center')
    axes[0, 0].set_axis_off()

# Composite Risk histogram
if 'Composite_Risk' in df.columns:
    df['Composite_Risk'].dropna().hist(bins=50, ax=axes[0, 1], edgecolor='black')
    axes[0, 1].set_title('Composite Risk Score Distribution', fontweight='bold')
    axes[0, 1].set_xlabel('Risk Score (0-10)')
    axes[0, 1].set_ylabel('Frequency')
else:
    axes[0, 1].text(0.5, 0.5, 'Composite_Risk missing', ha='center', va='center')
    axes[0, 1].set_axis_off()

# Trend direction pie
if 'Trend_Direction' in df.columns:
    df['Trend_Direction'].value_counts().plot(
        kind='pie',
        ax=axes[1, 0],
        autopct='%1.1f%%'
    )
    axes[1, 0].set_title('Trend Direction Distribution', fontweight='bold')
    axes[1, 0].set_ylabel('')
else:
    axes[1, 0].text(0.5, 0.5, 'Trend_Direction missing', ha='center', va='center')
    axes[1, 0].set_axis_off()

# Monthly anomaly trend
monthly_trend = (
    df.groupby('Year_Month')['Anomaly_Ensemble']
    .sum()
    .dropna()
)

if monthly_trend.empty:
    axes[1, 1].text(0.5, 0.5, 'No anomaly trend data', ha='center', va='center')
    axes[1, 1].set_axis_off()
else:
    monthly_trend.plot(ax=axes[1, 1], marker='o')
    axes[1, 1].set_title('Anomalies Over Time', fontweight='bold')
    axes[1, 1].set_xlabel('Date')
    axes[1, 1].set_ylabel('Anomaly Count')
    axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('risk_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Visualizations created safely!")


In [ ]:
# ============================================================================
# STEP 11: SAVE RESULTS (RECTIFIED & FAIL-SAFE)
# ============================================================================

from datetime import datetime
import pandas as pd

print("\n" + "="*80)
print("💾 STEP 11: SAVING RESULTS")
print("="*80)

# ----------------------------------------------------------------------------
# Ensure Year_Month is datetime
# ----------------------------------------------------------------------------
df['Year_Month'] = pd.to_datetime(df['Year_Month'], errors='coerce')

# ----------------------------------------------------------------------------
# 1️⃣ Full analysis
# ----------------------------------------------------------------------------
df.to_csv('full_analysis.csv', index=False)
print("✅ Saved: full_analysis.csv")

# ----------------------------------------------------------------------------
# 2️⃣ Anomalies only
# ----------------------------------------------------------------------------
anomalies = df[df['Anomaly_Ensemble'] == True]
anomalies.to_csv('detected_anomalies.csv', index=False)
print(f"✅ Saved: detected_anomalies.csv ({len(anomalies):,} anomalies)")

# ----------------------------------------------------------------------------
# 3️⃣ Top 10 anomalies (safe recompute)
# ----------------------------------------------------------------------------
top_anomalies = (
    df[df['Anomaly_Ensemble']]
    .nlargest(10, 'Z_Score')
)

top_anomalies.to_csv('top_10_anomalies.csv', index=False)
print("✅ Saved: top_10_anomalies.csv")

# ----------------------------------------------------------------------------
# 4️⃣ Critical cases
# ----------------------------------------------------------------------------
if 'Priority' in df.columns:
    critical = df[df['Priority'] == 'CRITICAL']
else:
    critical = pd.DataFrame()

critical.to_csv('critical_cases.csv', index=False)
print(f"✅ Saved: critical_cases.csv ({len(critical):,} cases)")

# ----------------------------------------------------------------------------
# 5️⃣ District summary
# ----------------------------------------------------------------------------
district_summary = (
    df.groupby(['State', 'District'])
    .agg({
        'Composite_Risk': 'mean',
        'DPI': 'mean',
        'OSI': 'mean',
        'UAG': 'mean',
        'Anomaly_Ensemble': 'sum'
    })
    .round(2)
    .sort_values('Composite_Risk', ascending=False)
)

district_summary.to_csv('district_summary.csv')
print("✅ Saved: district_summary.csv")

# ----------------------------------------------------------------------------
# 6️⃣ Executive Summary (FULLY SAFE)
# ----------------------------------------------------------------------------
with open('executive_summary.txt', 'w') as f:
    f.write("=" * 80 + "\n")
    f.write("UIDAI AADHAAR INTELLIGENCE SYSTEM - EXECUTIVE SUMMARY\n")
    f.write("=" * 80 + "\n\n")

    f.write(f"Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

    # SAFE DATE RANGE
    min_date = df['Year_Month'].min()
    max_date = df['Year_Month'].max()

    min_date_str = min_date.strftime('%B %Y') if pd.notna(min_date) else "Unknown"
    max_date_str = max_date.strftime('%B %Y') if pd.notna(max_date) else "Unknown"

    f.write(f"Data Period: {min_date_str} to {max_date_str}\n")
    f.write(f"Total Records Analyzed: {len(df):,}\n\n")

    # KEY FINDINGS
    anomaly_count = int(df['Anomaly_Ensemble'].sum())
    f.write("KEY FINDINGS:\n\n")

    f.write("1. ANOMALY DETECTION\n")
    f.write(f"   - Total Anomalies: {anomaly_count:,} ({(anomaly_count / len(df) * 100):.2f}%)\n")
    f.write(f"   - Critical Cases: {len(critical):,}\n\n")

    # Geographic distribution
    state_anomalies = (
        df[df['Anomaly_Ensemble']]
        .groupby('State')
        .size()
        .sort_values(ascending=False)
    )

    f.write("2. GEOGRAPHIC DISTRIBUTION\n")
    if not state_anomalies.empty:
        f.write(
            f"   - Most Affected State: "
            f"{state_anomalies.index[0]} "
            f"({state_anomalies.iloc[0]:,} anomalies)\n"
        )
    else:
        f.write("   - Most Affected State: None\n")

    f.write("   - Top 3 Risky Districts:\n")
    if not district_summary.empty:
        for i, ((state, district), row) in enumerate(district_summary.head(3).iterrows(), 1):
            f.write(f"     {i}. {district}, {state} (Risk: {row['Composite_Risk']}/10)\n")
    else:
        f.write("     No high-risk districts identified\n")

    f.write("\n")

    # Temporal patterns
    monthly_anomalies = (
        df.groupby(df['Year_Month'].dt.month_name())['Anomaly_Ensemble']
        .sum()
    )

    f.write("3. TEMPORAL PATTERNS\n")
    if not monthly_anomalies.empty:
        f.write(
            f"   - Peak Anomaly Month: "
            f"{monthly_anomalies.idxmax()} "
            f"({monthly_anomalies.max():,} anomalies)\n\n"
        )
    else:
        f.write("   - Peak Anomaly Month: None\n\n")

    # Trend direction
    f.write("4. TRENDS\n")
    if 'Trend_Direction' in df.columns:
        trend_dist = df['Trend_Direction'].value_counts()
        for direction, count in trend_dist.items():
            f.write(f"   - {direction}: {count:,} records\n")
    else:
        f.write("   - Trend data unavailable\n")

print("✅ Saved: executive_summary.txt")

# ----------------------------------------------------------------------------
# 7️⃣ OPTIONAL DOWNLOADS (COLAB-SAFE)
# ----------------------------------------------------------------------------
try:
    from google.colab import files
    print("\n📥 Downloading all result files...")
    for file in [
        'full_analysis.csv',
        'detected_anomalies.csv',
        'top_10_anomalies.csv',
        'critical_cases.csv',
        'district_summary.csv',
        'executive_summary.txt',
        'anomaly_heatmap.png',
        'risk_analysis.png'
    ]:
        files.download(file)
except Exception:
    print("ℹ️ File download skipped (not running in Colab)")


In [ ]:
# ============================================================================
# FINAL SUMMARY
# ============================================================================

print("\n" + "="*80)
print("✅ ANALYSIS COMPLETE!")
print("="*80)

print(f"\n📊 FINAL STATISTICS:")
print(f"   Total Records: {len(df):,}")
print(f"   Anomalies Detected: {anomaly_count:,} ({(anomaly_count/len(df)*100):.2f}%)")
print(f"   Critical Cases: {len(critical):,}")
print(f"   High Priority: {len(df[df['Priority'] == 'HIGH']):,}")
print(f"   Districts Analyzed: {df.groupby(['State', 'District']).ngroups:,}")

print(f"\n📁 GENERATED FILES:")
print(f"   1. full_analysis.csv - Complete dataset with all results")
print(f"   2. detected_anomalies.csv - Only anomalous records")
print(f"   3. top_10_anomalies.csv - Top 10 shocking findings")
print(f"   4. critical_cases.csv - Urgent cases requiring action")
print(f"   5. district_summary.csv - Risk scores by district")
print(f"   6. executive_summary.txt - Key findings summary")
print(f"   7. anomaly_heatmap.png - Visual heatmap")
print(f"   8. risk_analysis.png - Risk distribution charts")

print("\n🎯 USE THESE FINDINGS IN YOUR HACKATHON SUBMISSION!")
print("="*80)

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


In [ ]:
import shutil

shutil.move(
    '/content/full_analysis.csv',
    '/content/drive/MyDrive/full_analysis.csv'
)


In [ ]:
# ============================================================
# SCHEMA ALIGNMENT HELPERS (DO NOT CHANGE ML LOGIC)
# ============================================================

def normalize_metric_category(x):
    mapping = {
        'ENROLMENT': 'enrolment',
        'BIOMETRIC_UPDATE': 'biometric_update',
        'DEMOGRAPHIC_UPDATE': 'demographic_update'
    }
    return mapping.get(str(x).upper(), 'enrolment')


def normalize_age_group(x):
    mapping = {
        '0-5': 'age_0_5',
        '6-17': 'age_6_17',
        '18+': 'age_18_plus'
    }
    return mapping.get(str(x), 'age_18_plus')


def normalize_severity(x):
    mapping = {
        'LOW': 'low',
        'MEDIUM': 'medium',
        'HIGH': 'high',
        'CRITICAL': 'critical'
    }
    return mapping.get(str(x).upper(), 'medium')


def normalize_trend(x):
    mapping = {
        'INCREASING': 'increasing',
        'DECREASING': 'decreasing',
        'STABLE': 'stable'
    }
    return mapping.get(str(x).upper(), 'stable')


def normalize_pattern(x):
    mapping = {
        'SEASONAL_SPIKE': 'seasonal',
        'NONE': 'none'
    }
    return mapping.get(str(x).upper(), 'none')


In [ ]:
derived_metrics = df.copy()

derived_metrics_df = pd.DataFrame({
    "year": derived_metrics["Year"],
    "month": derived_metrics["Month"],
    "state": derived_metrics["State"],
    "district": derived_metrics["District"],
    "metricCategory": derived_metrics["Metric_Type"].apply(normalize_metric_category),
    "ageGroup": derived_metrics["Age_Group"].apply(normalize_age_group),

    "growthRate": derived_metrics["Deviation_Pct"] / 100,
    "deviationFromBaseline": derived_metrics["Deviation"],
    "spikeRatio": abs(derived_metrics["Z_Score"]),
    "volatility": derived_metrics["Baseline_Std"],

    "demandPressureIndex": derived_metrics["DPI"],
    "operationalStressIndex": derived_metrics["OSI"],
    "updateAccessibilityGap": derived_metrics["UAG"],
    "compositeRiskScore": derived_metrics["Composite_Risk"],

    "sourceBatchId": "batch_v1"
})

derived_metrics_df.to_csv("derived_metrics.csv", index=False)
print("✅ derived_metrics.csv generated")


In [ ]:
anomaly_df = df[df["Anomaly_Ensemble"] == True].copy()

anomaly_results = pd.DataFrame({
    "year": anomaly_df["Year"],
    "month": anomaly_df["Month"],
    "state": anomaly_df["State"],
    "district": anomaly_df["District"],
    "metricCategory": anomaly_df["Metric_Type"].apply(normalize_metric_category),
    "ageGroup": anomaly_df["Age_Group"].apply(normalize_age_group),

    "isAnomaly": True,
    "anomalyScore": anomaly_df["Z_Score"].abs(),
    "anomalySeverity": anomaly_df["Severity_Level"].apply(normalize_severity),
    "anomalyConfidence": anomaly_df["Anomaly_Severity"],

    "sourceBatchId": "batch_v1"
})

anomaly_results.to_csv("anomaly_results.csv", index=False)
print("✅ anomaly_results.csv generated")


In [ ]:
trend_df = df[df["Trend_Direction"] != "STABLE"].copy()

trend_results = pd.DataFrame({
    "year": trend_df["Year"],
    "month": trend_df["Month"],
    "state": trend_df["State"],
    "district": trend_df["District"],
    "metricCategory": trend_df["Metric_Type"].apply(normalize_metric_category),
    "ageGroup": trend_df["Age_Group"].apply(normalize_age_group),

    "trendDirection": trend_df["Trend_Direction"].apply(normalize_trend),
    "trendSlope": trend_df["Trend_Slope"],
    "trendStrength": trend_df["Trend_RSquared"],
    "trendConfidence": trend_df["Trend_RSquared"],

    "sourceBatchId": "batch_v1"
})

trend_results.to_csv("trend_results.csv", index=False)
print("✅ trend_results.csv generated")


In [ ]:
pattern_df = df[df["Pattern_Seasonal"] == True].copy()

pattern_results = pd.DataFrame({
    "year": pattern_df["Year"],
    "month": pattern_df["Month"],
    "state": pattern_df["State"],
    "district": pattern_df["District"],
    "metricCategory": pattern_df["Metric_Type"].apply(normalize_metric_category),
    "ageGroup": pattern_df["Age_Group"].apply(normalize_age_group),

    "hasPattern": True,
    "dominantPatternType": pattern_df["Pattern_Type"].apply(normalize_pattern),
    "patternStrength": 0.7,
    "patternConfidence": 0.75,

    "sourceBatchId": "batch_v1"
})

pattern_results.to_csv("pattern_results.csv", index=False)
print("✅ pattern_results.csv generated")


In [ ]:
predictive = df.copy()

predictive_results = pd.DataFrame({
    "year": predictive["Year"],
    "month": predictive["Month"],
    "state": predictive["State"],
    "district": predictive["District"],
    "metricCategory": predictive["Metric_Type"].apply(normalize_metric_category),
    "ageGroup": predictive["Age_Group"].apply(normalize_age_group),

    "riskSignal": predictive["Priority"].map({
        "LOW": "stable",
        "MEDIUM": "risk_building",
        "HIGH": "likely_spike",
        "CRITICAL": "likely_spike"
    }),

    "riskScore": predictive["Composite_Risk"],
    "predictionConfidence": 0.7,
    "contributingFactors": "DPI, OSI, UAG",

    "sourceBatchId": "batch_v1"
})

predictive_results.to_csv("predictive_indicators.csv", index=False)
print("✅ predictive_indicators.csv generated")


In [ ]:
solution_results = pd.DataFrame({
    "year": df["Year"],
    "month": df["Month"],
    "state": df["State"],
    "district": df["District"],
    "metricCategory": df["Metric_Type"].apply(normalize_metric_category),
    "ageGroup": df["Age_Group"].apply(normalize_age_group),

    "frameworkType": df["Solution_Framework"].str.lower(),
    "frameworkConfidence": 0.8,
    "rationale": df["Recommended_Actions"],
    "drivingIndexes": "DPI, OSI, UAG",
    "predictiveSignal": df["Priority"].map({
        "LOW": "stable",
        "MEDIUM": "risk_building",
        "HIGH": "likely_spike",
        "CRITICAL": "likely_spike"
    }),

    "sourceBatchId": "batch_v1"
})

solution_results.to_csv("solution_frameworks.csv", index=False)
print("✅ solution_frameworks.csv generated")
